# Generating COMBINE Archives from the SBML Test Suite

The [SBML test suite](http://sbml.org/Software/SBML_Test_Suite) is intended to test the accuracy SBML ODE simulators by providing a normative standard to compare simulation results to. The test suite predates SED-ML and specifies the simulation parameters in a custom format. However, recent versions include the SED-ML necessary to reproduce the simulations. Here, we provide an automated script to convert these test cases into COMBINE archives, which can be imported in Tellurium.

## Step 1: Download the SBML test cases